<a href="https://colab.research.google.com/github/AliKarimiENT/NLP_HW1/blob/main/NLPHW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reading txt file and getting verses

In [17]:
# with this function we check that it contains number in the text or not
def hasNumbers(inputString) :
  for character in inputString :
    if character.isdigit() :
      return True
  return False



with open('/content/ShamsDaftar6.doc_filename=ShamsDaftar6.txt') as f:
  lines = f.readlines()

verses = []
for line in lines :
  if line != ' ' and line != '\n' and hasNumbers(line) == False and :
    verses.append(line.replace('\t',' '))

for i in range(20) :
  print(verses[i])

﻿دیوان شمس تبریزی (غزلیات)

--------------------------------------------------------

گرم سیم و درم بودی مرا مونس چه کم بودی وگر یارم فقیرستی ز زر فارغ چه غم بودی

خدایا حرمت مردان ز دنیا فارغش گردان از آن گر فارغستی او ز پیش من چه کم بودی

نگارا گر مرا خواهی وگر همدرد و همراهی مکن آه و مخور حسرت که بختم محتشم بودی

بتا زیبا و نیکویی رها کن این گدارویی اگر چشم تو سیرستی فلک ما را حشم بودی

ز طمع آدمی باشد که خویش از وی چو بیگانه است وگر او بی طمع بودی همه کس خال و عم بودی

بیا چون ما شو ای مه رو نه نعمت جو نه دولت جو گر ابلیس این چنین بودی شه و صاحب علم بودی

از ابلیسی جدا بودی سقط او را ثنا بودی جفا او را وفا بودی سقم او را کرم بودی

زهی اقبال درویشی زهی اسرار بی خویشی اگر دانستیی پیشت همه هستی عدم بودی

جهانی هیچ و ما هیچان خیال و خواب ما پیچان وگر خفته بدانستی که در خوابم چه غم بودی

خیالی بیند این خفته در اندیشه فرورفته وگر زین خواب آشفته بجستی در نعم بودی

یکی زندان غم دیده یکی باغ ارم دیده وگر بیدار گشتی او نه زندان نی ارم بودی

امیر دل همی گوید تو را گر تو دلی داری که عاشق باش ت